In [1]:
import pandas as pd
import numpy as np
import functions
from functions import UniprotDataFetcher
from functions import PDB_analyzer
import requests
import ast
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [24]:
# upload fully compiled dataset
df_full = pd.read_excel("FullDataSetPDB_Filed_Ordered.xlsx")
print(f"Length of dataset: {len(df_full)}")
df_full.head()

Length of dataset: 7816


,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD']
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['6TAV', '7O7L', '7O7M', '7VON', '7O7Q', '7O7R..."
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q1Y', '7Q5Z', '7Q62', '7Q60', '7Q61']"
3,Q9NRG9,reviewed,AAAS_HUMAN,Aladin (Adracalin),AAAS ADRACALA GL003,Homo sapiens (Human),546,NaN
4,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS..."


In [25]:
# Get all cases where PDB_FILES_Ordered = Null
null_cases = df_full[df_full["PDB_Files_Ordered"].isnull()]
print(f"Number of null cases: {len(null_cases)}")
# Remove any null cases from the full com
df_full = df_full.dropna(subset=['PDB_Files_Ordered'])
# Reset the index after removing rows
df_full = df_full.reset_index(drop=True)
print(f"Length of full dataset: {len(df_full)}")
# store null cases into an excel file
null_cases.to_excel("NullCases.xlsx", index=False) 

Number of null cases: 210
Length of full dataset: 7606


In [26]:
# make sure all NaN cases removed
df_full

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD']
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['6TAV', '7O7L', '7O7M', '7VON', '7O7Q', '7O7R..."
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q1Y', '7Q5Z', '7Q62', '7Q60', '7Q61']"
3,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS..."
4,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']"
...,...,...,...,...,...,...,...,...
7601,P0DOX7,reviewed,IGK_HUMAN,Immunoglobulin kappa light chain (Immunoglobul...,NaN,Homo sapiens (Human),214,"['6N35', '6UOE', '6MUB', '6UTE', '6N2X', '5VIG..."
7602,P31785,reviewed,IL2RG_HUMAN,Cytokine receptor common subunit gamma (Interl...,IL2RG,Homo sapiens (Human),369,"['7S2R', '4GS7', '2ERJ', '3BPL', '3QB7', '8EPA..."
7603,P08700,reviewed,IL3_HUMAN,Interleukin-3 (IL-3) (Hematopoietic growth fac...,IL3,Homo sapiens (Human),152,"['5UWC', '6NMY', '5UV8']"
7604,O95760,reviewed,IL33_HUMAN,Interleukin-33 (IL-33) (Interleukin-1 family m...,IL33 C9orf26 IL1F11 NFHEV,Homo sapiens (Human),270,['4KC3']


In [11]:
# Function that gets first element from list that is oriented in the matter of PDB_Files_Ordered
def get_first_element(stringed_list):
    # Remove whitespace and convert to lowercase
    stringed_list = stringed_list.strip().lower()

    # Check if the string starts with '[' and ends with ']'
    if stringed_list.startswith('[') and stringed_list.endswith(']'):
        # Remove the brackets
        stringed_list = stringed_list[1:-1]

    # Remove any single quotes and double quotes
    stringed_list = stringed_list.replace("'", "").replace('"', '')

    # Split the string into individual elements
    elements = stringed_list.split(',')

    # Get the first element
    first_element = elements[0].strip().upper()

    return first_element

# Make a new column that applies this function to every row of the df on the column pdb_files_ordered
df_full['BestRanked'] = df_full['PDB_Files_Ordered'].apply(get_first_element)
# check
df_full

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['6TAV', '7O7L', '7O7M', '7VON', '7O7Q', '7O7R...",6TAV
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q1Y', '7Q5Z', '7Q62', '7Q60', '7Q61']",7Q1Y
3,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A
4,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0
...,...,...,...,...,...,...,...,...,...
7601,P0DOX7,reviewed,IGK_HUMAN,Immunoglobulin kappa light chain (Immunoglobul...,NaN,Homo sapiens (Human),214,"['6N35', '6UOE', '6MUB', '6UTE', '6N2X', '5VIG...",6N35
7602,P31785,reviewed,IL2RG_HUMAN,Cytokine receptor common subunit gamma (Interl...,IL2RG,Homo sapiens (Human),369,"['7S2R', '4GS7', '2ERJ', '3BPL', '3QB7', '8EPA...",7S2R
7603,P08700,reviewed,IL3_HUMAN,Interleukin-3 (IL-3) (Hematopoietic growth fac...,IL3,Homo sapiens (Human),152,"['5UWC', '6NMY', '5UV8']",5UWC
7604,O95760,reviewed,IL33_HUMAN,Interleukin-33 (IL-33) (Interleukin-1 family m...,IL33 C9orf26 IL1F11 NFHEV,Homo sapiens (Human),270,['4KC3'],4KC3


In [12]:
df_full["Chain"] = df_full.apply(lambda row: UniprotDataFetcher(row["Entry"]).get_chain_only(row["BestRanked"]), axis=1)

Chain:A=223-308
Chain:A/B/C/D=1-1474
Chain:A/B=19-1454
Chain:A=1-425
Chain:A=27-365
Chain:A/B=2-122
Chain:E/F/G/H=197-207
Chain:A=1-384
Chain:A/B/C/D/E/F/G/H=4-453
Chain:A/B=783-985
Chain:A=14-309
Chain:A=455-926
Chain:E/F=539-547
Chain:A=1-2261
Chain:A=1-1704
Chain:A=1-2273
Chain:A=32-1279
Chain:A=152-738
Chain:U=1-1321
Chain:A=1-1286
Chain:A/B=1-842
Chain:A/B=72-751
Chain:A/B/C/D/E/F/G/H=110-714
Chain:A=642-871
Chain:A=1254-1503
Chain:E=2-1581
Chain:A/B=65-745
Chain:A/B=2-606
Chain:A/B=300-841
Chain:A/D=1-678
Chain:A/B=1-655
Chain:A/C=1-651
Chain:B/D=2-673
Chain:A=1-210
Chain:A/B=43-337
Chain:A=407-442
Chain:F=1-154
Chain:A=38-512
Chain:A=279-546
Chain:A=73-140
Chain:A=609-683
Chain:A=64-354
Chain:A=1-81
Chain:B=399-409
Chain:A/F=1-415
Chain:A/B=30-423
Chain:A/B/C/D=1-397
Chain:A/B/C/D/E/F=1571-2338
Chain:A/B/C/D=1693-2450
Chain:A/B=355-780
Chain:A/B/C/D=24-415
Chain:A/B/C/D=35-421
Chain:A/B/C/D/E/F/G/H=30-412
Chain:A/B/C/D=52-432
Chain:A=72-655
Chain:C=89-103
Chain:A/B/C/D=1-377
Cha

In [13]:
df_full

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD,A=223-308
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['6TAV', '7O7L', '7O7M', '7VON', '7O7Q', '7O7R...",6TAV,A/B/C/D=1-1474
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q1Y', '7Q5Z', '7Q62', '7Q60', '7Q61']",7Q1Y,A/B=19-1454
3,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A,A=1-425
4,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0,A=27-365
...,...,...,...,...,...,...,...,...,...,...
7601,P0DOX7,reviewed,IGK_HUMAN,Immunoglobulin kappa light chain (Immunoglobul...,NaN,Homo sapiens (Human),214,"['6N35', '6UOE', '6MUB', '6UTE', '6N2X', '5VIG...",6N35,K/L=1-213
7602,P31785,reviewed,IL2RG_HUMAN,Cytokine receptor common subunit gamma (Interl...,IL2RG,Homo sapiens (Human),369,"['7S2R', '4GS7', '2ERJ', '3BPL', '3QB7', '8EPA...",7S2R,A=54-254
7603,P08700,reviewed,IL3_HUMAN,Interleukin-3 (IL-3) (Hematopoietic growth fac...,IL3,Homo sapiens (Human),152,"['5UWC', '6NMY', '5UV8']",5UWC,I=31-152
7604,O95760,reviewed,IL33_HUMAN,Interleukin-33 (IL-33) (Interleukin-1 family m...,IL33 C9orf26 IL1F11 NFHEV,Homo sapiens (Human),270,['4KC3'],4KC3,A=112-270


In [14]:
df_full
df_full.to_excel("Uniport_human_AF2_Ranked_Chains.xlsx", index=False)

In [17]:
df_full_demo = df_full.head(4)

In [18]:
df_full_demo["Chain"] = df_full_demo.apply(lambda row: UniprotDataFetcher(row["Entry"]).get_chain_only(row["BestRanked"]), axis=1)

/tmp/ipykernel_3803218/3261412456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full_demo["Chain"] = df_full_demo.apply(lambda row: UniprotDataFetcher(row["Entry"]).get_chain_only(row["BestRanked"]), axis=1)


In [ ]:
df_full_demo

## Run TWO

In [17]:
# import approp dfs
df_1 = pd.read_excel("secondrun1.xlsx")
df_2 = pd.read_excel("secondrun2.xlsx") 
df_3 = pd.read_excel("secondrun3.xlsx")
df_4 = pd.read_excel("secondrun4.xlsx")
df_5 = pd.read_excel("secondrun5.xlsx")
df_6 = pd.read_excel("secondrun6.xlsx")

In [19]:
comb_df_2 =  pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True)
comb_df_2.drop_duplicates(inplace=True)

In [20]:
comb_df_2

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD']
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['7O7Q', '7O7R', '6TAV', '7O7L', '7O7M', '7VOO..."
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q5Z', '7Q62', '7Q60', '7Q61', '7Q1Y']"
3,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS..."
4,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']"
...,...,...,...,...,...,...,...,...
7601,O00370,reviewed,LORF2_HUMAN,LINE-1 retrotransposable element ORF2 protein ...,NaN,Homo sapiens (Human),1275,"['1VYB', '2V0R', '7N8K', '7N8S', '7N94', '2V0S']"
7602,P0DOX3,reviewed,IGD_HUMAN,Immunoglobulin delta heavy chain (Immunoglobul...,NaN,Homo sapiens (Human),512,['1ZVO']
7603,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT..."
7604,P0DOX8,reviewed,IGL1_HUMAN,Immunoglobulin lambda-1 light chain (Immunoglo...,NaN,Homo sapiens (Human),216,"['1MCE', '1MCD']"


In [21]:
# Function that gets first element from list that is oriented in the matter of PDB_Files_Ordered
def get_first_element(stringed_list):
    # Remove whitespace and convert to lowercase
    stringed_list = stringed_list.strip().lower()

    # Check if the string starts with '[' and ends with ']'
    if stringed_list.startswith('[') and stringed_list.endswith(']'):
        # Remove the brackets
        stringed_list = stringed_list[1:-1]

    # Remove any single quotes and double quotes
    stringed_list = stringed_list.replace("'", "").replace('"', '')

    # Split the string into individual elements
    elements = stringed_list.split(',')

    # Get the first element
    first_element = elements[0].strip().upper()

    return first_element

In [23]:
# Make a new column that applies this function to every row of the df on the column pdb_files_ordered
comb_df_2['BestRanked'] = comb_df_2['PDB_Files_Ordered'].apply(get_first_element)
# check
comb_df_2

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['7O7Q', '7O7R', '6TAV', '7O7L', '7O7M', '7VOO...",7O7Q
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q5Z', '7Q62', '7Q60', '7Q61', '7Q1Y']",7Q5Z
3,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A
4,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0
...,...,...,...,...,...,...,...,...,...
7601,O00370,reviewed,LORF2_HUMAN,LINE-1 retrotransposable element ORF2 protein ...,NaN,Homo sapiens (Human),1275,"['1VYB', '2V0R', '7N8K', '7N8S', '7N94', '2V0S']",1VYB
7602,P0DOX3,reviewed,IGD_HUMAN,Immunoglobulin delta heavy chain (Immunoglobul...,NaN,Homo sapiens (Human),512,['1ZVO'],1ZVO
7603,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT...",8DV1
7604,P0DOX8,reviewed,IGL1_HUMAN,Immunoglobulin lambda-1 light chain (Immunoglo...,NaN,Homo sapiens (Human),216,"['1MCE', '1MCD']",1MCE


In [28]:
# import old df
old_df = pd.read_excel("Uniport_human_AF2_Ranked_Chains.xlsx")
old_df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD,A=223-308
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['6TAV', '7O7L', '7O7M', '7VON', '7O7Q', '7O7R...",6TAV,A/B/C/D=1-1474
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q1Y', '7Q5Z', '7Q62', '7Q60', '7Q61']",7Q1Y,A/B=19-1454
3,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A,A=1-425
4,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0,A=27-365
...,...,...,...,...,...,...,...,...,...,...
7601,P0DOX7,reviewed,IGK_HUMAN,Immunoglobulin kappa light chain (Immunoglobul...,NaN,Homo sapiens (Human),214,"['6N35', '6UOE', '6MUB', '6UTE', '6N2X', '5VIG...",6N35,K/L=1-213
7602,P31785,reviewed,IL2RG_HUMAN,Cytokine receptor common subunit gamma (Interl...,IL2RG,Homo sapiens (Human),369,"['7S2R', '4GS7', '2ERJ', '3BPL', '3QB7', '8EPA...",7S2R,A=54-254
7603,P08700,reviewed,IL3_HUMAN,Interleukin-3 (IL-3) (Hematopoietic growth fac...,IL3,Homo sapiens (Human),152,"['5UWC', '6NMY', '5UV8']",5UWC,I=31-152
7604,O95760,reviewed,IL33_HUMAN,Interleukin-33 (IL-33) (Interleukin-1 family m...,IL33 C9orf26 IL1F11 NFHEV,Homo sapiens (Human),270,['4KC3'],4KC3,A=112-270


In [52]:
merge_df_og = pd.merge(comb_df_2, old_df) 
merge_df_og

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD,A=223-308
1,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A,A=1-425
2,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0,A=27-365
3,Q9H7C9,reviewed,AAMDC_HUMAN,Mth938 domain-containing protein (Adipogenesis...,AAMDC C11orf67 PTD015,Homo sapiens (Human),122,"['2Q4Q', '2AB1']",2Q4Q,A/B=2-122
4,Q16613,reviewed,SNAT_HUMAN,Serotonin N-acetyltransferase (Serotonin acety...,AANAT SNAT,Homo sapiens (Human),207,['6T80'],6T80,E/F/G/H=197-207
...,...,...,...,...,...,...,...,...,...,...
6796,Q9C0D3,reviewed,ZY11B_HUMAN,Protein zyg-11 homolog B,ZYG11B KIAA1730,Homo sapiens (Human),744,"['7EP2', '7EP0', '7XYV', '7EP1', '7XYW', '7XYX...",7EP2,A/B/C/D=443-728
6797,Q8IYH5,reviewed,ZZZ3_HUMAN,ZZ-type zinc finger-containing protein 3,ZZZ3,Homo sapiens (Human),903,"['2FC7', '2YUM', '6E83', '6E86']",2FC7,A=807-886
6798,O00370,reviewed,LORF2_HUMAN,LINE-1 retrotransposable element ORF2 protein ...,NaN,Homo sapiens (Human),1275,"['1VYB', '2V0R', '7N8K', '7N8S', '7N94', '2V0S']",1VYB,A/B=1-238
6799,P0DOX3,reviewed,IGD_HUMAN,Immunoglobulin delta heavy chain (Immunoglobul...,NaN,Homo sapiens (Human),512,['1ZVO'],1ZVO,C/D=1-512


In [54]:
merge_df = pd.merge(comb_df_2, old_df, how='left', indicator=True)
leftover_df = merge_df[merge_df['_merge'] == 'left_only']
leftover_df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain,_merge
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['7O7Q', '7O7R', '6TAV', '7O7L', '7O7M', '7VOO...",7O7Q,NaN,left_only
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q5Z', '7Q62', '7Q60', '7Q61', '7Q1Y']",7Q5Z,NaN,left_only
9,Q5JTZ9,reviewed,SYAM_HUMAN,"Alanine--tRNA ligase, mitochondrial (EC 6.1.1....",AARS2 AARSL KIAA1270,Homo sapiens (Human),985,"['6NLY', '6NLQ', '6NOW']",6NLY,NaN,left_only
13,O95477,reviewed,ABCA1_HUMAN,Phospholipid-transporting ATPase ABCA1 (EC 7.6...,ABCA1 ABC1 CERP,Homo sapiens (Human),2261,"['7TBW', '5XJY', '7ROQ', '7TDT', '7TBY', '7TC0...",7TBW,NaN,left_only
16,P08183,reviewed,MDR1_HUMAN,ATP-dependent translocase ABCB1 (ATP-binding c...,ABCB1 MDR1 PGY1,Homo sapiens (Human),1280,"['6FN1', '6QEX', '7O9W', '7A69', '7A6F', '7A6C...",6FN1,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
7476,P12956,reviewed,XRCC6_HUMAN,X-ray repair cross-complementing protein 6 (EC...,XRCC6 G22P1,Homo sapiens (Human),609,"['1JEQ', '7SGL', '8AG4', '8AG5', '7AXZ', '6ERG...",1JEQ,NaN,left_only
7501,P63104,reviewed,1433Z_HUMAN,14-3-3 protein zeta/delta (Protein kinase C in...,YWHAZ,Homo sapiens (Human),245,"['4ZDR', '8AH2', '6ZFD', '6ZFG', '7Q16', '3RDH...",4ZDR,NaN,left_only
7524,Q6NZY4,reviewed,ZCHC8_HUMAN,Zinc finger CCHC domain-containing protein 8 (...,ZCCHC8,Homo sapiens (Human),707,"['7S7C', '7S7B', '7Z4Y', '7Z4Z', '6C90', '5LXR...",7S7C,NaN,left_only
7603,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT...",8DV1,NaN,left_only


In [55]:
# Assuming you already have the leftover_df DataFrame from the previous code

# Selecting specific columns from the leftover_df DataFrame
selected_columns = ['Entry', 'Reviewed', 'Entry Name', 'Protein names', 'Gene Names',
                    'Organism', 'Length', 'PDB_Files_Ordered', 'BestRanked']

# Creating a new DataFrame with only the selected columns
filtered_leftover_df = leftover_df[selected_columns]


In [62]:
leftover_df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain,_merge
1,P01023,reviewed,A2MG_HUMAN,Alpha-2-macroglobulin (Alpha-2-M) (C3 and PZP-...,A2M CPAMD5 FWP007,Homo sapiens (Human),1474,"['7O7Q', '7O7R', '6TAV', '7O7L', '7O7M', '7VOO...",7O7Q,NaN,left_only
2,A8K2U0,reviewed,A2ML1_HUMAN,Alpha-2-macroglobulin-like protein 1 (C3 and P...,A2ML1 CPAMD9,Homo sapiens (Human),1454,"['7Q5Z', '7Q62', '7Q60', '7Q61', '7Q1Y']",7Q5Z,NaN,left_only
9,Q5JTZ9,reviewed,SYAM_HUMAN,"Alanine--tRNA ligase, mitochondrial (EC 6.1.1....",AARS2 AARSL KIAA1270,Homo sapiens (Human),985,"['6NLY', '6NLQ', '6NOW']",6NLY,NaN,left_only
13,O95477,reviewed,ABCA1_HUMAN,Phospholipid-transporting ATPase ABCA1 (EC 7.6...,ABCA1 ABC1 CERP,Homo sapiens (Human),2261,"['7TBW', '5XJY', '7ROQ', '7TDT', '7TBY', '7TC0...",7TBW,NaN,left_only
16,P08183,reviewed,MDR1_HUMAN,ATP-dependent translocase ABCB1 (ATP-binding c...,ABCB1 MDR1 PGY1,Homo sapiens (Human),1280,"['6FN1', '6QEX', '7O9W', '7A69', '7A6F', '7A6C...",6FN1,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
7476,P12956,reviewed,XRCC6_HUMAN,X-ray repair cross-complementing protein 6 (EC...,XRCC6 G22P1,Homo sapiens (Human),609,"['1JEQ', '7SGL', '8AG4', '8AG5', '7AXZ', '6ERG...",1JEQ,NaN,left_only
7501,P63104,reviewed,1433Z_HUMAN,14-3-3 protein zeta/delta (Protein kinase C in...,YWHAZ,Homo sapiens (Human),245,"['4ZDR', '8AH2', '6ZFD', '6ZFG', '7Q16', '3RDH...",4ZDR,NaN,left_only
7524,Q6NZY4,reviewed,ZCHC8_HUMAN,Zinc finger CCHC domain-containing protein 8 (...,ZCCHC8,Homo sapiens (Human),707,"['7S7C', '7S7B', '7Z4Y', '7Z4Z', '6C90', '5LXR...",7S7C,NaN,left_only
7603,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT...",8DV1,NaN,left_only


In [63]:
leftover_df["Chain"] = leftover_df.apply(lambda row: UniprotDataFetcher(row["Entry"]).get_chain_only(row["BestRanked"]), axis=1)

Chain:A/B/C/D=1-1474
Chain:A=19-1454
Chain:A/B/C/D=802-985
Chain:A=1-2261
Chain:A=32-1279
Chain:A/B=1-842
Chain:E=2-1581
Chain:A/B=65-745
Chain:A/B=1-655
Chain:A/C=1-651
Chain:B/D=2-673
Chain:A/B/C/D/E/F=1571-2338
Chain:A/B=1-377
Chain:A=30-1240
Chain:B/D=2-805
Chain:A/B/C/D/E/F/G/H=1-424, A/B/C/D/E/F/G/H=488-1101
Chain:A/B/C/D/E/F=1-375
Chain:A/B/C/D/E=2-375
Chain:N=1-429
Chain:A/B=233-736
Chain:A=2-208, A=222-316
Chain:A=35-460
Chain:R=9-450
Chain:A=899-1163
Chain:A=34-485
Chain:O/P=114-207
Chain:A/B/C/D=678-1030
Chain:A/B/C/D=85-152
Chain:L=1-185
Chain:C=1-84
Chain:D=1-121
Chain:H=1-110
Chain:N=1-822
Chain:I=1-808
Chain:Y/Z=1-565
Chain:X=280-498
Chain:A=1-1881
Chain:A/B=1-660
Chain:A=40-217
Chain:S=1-154
Chain:B=1-651
Chain:A/C/E/G/I/K/M=1-1248
Chain:B=7-229, B=243-330
Chain:A/B/C/D/E/F/G/H=672-713
Chain:Q=1-1485
Chain:A/B=11-181
Chain:E=11-175
Chain:A/B=1-322
Chain:D/E=1-204
Chain:Q=1-689
Chain:D=1-300
Chain:G=1-151
Chain:C=2-376
Chain:A/D=35-294
Chain:A/B=370-496, A/B=539-617
Chai

/tmp/ipykernel_3214166/470023035.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leftover_df["Chain"] = leftover_df.apply(lambda row: UniprotDataFetcher(row["Entry"]).get_chain_only(row["BestRanked"]), axis=1)


In [74]:
selected_columns = ['Entry', 'Reviewed', 'Entry Name', 'Protein names', 'Gene Names',
                    'Organism', 'Length', 'PDB_Files_Ordered', 'BestRanked', "Chain"]
leftover_df = leftover_df[selected_columns]

In [76]:
comb_df = pd.concat([merge_df_og,leftover_df])
comb_df

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD,A=223-308
1,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A,A=1-425
2,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0,A=27-365
3,Q9H7C9,reviewed,AAMDC_HUMAN,Mth938 domain-containing protein (Adipogenesis...,AAMDC C11orf67 PTD015,Homo sapiens (Human),122,"['2Q4Q', '2AB1']",2Q4Q,A/B=2-122
4,Q16613,reviewed,SNAT_HUMAN,Serotonin N-acetyltransferase (Serotonin acety...,AANAT SNAT,Homo sapiens (Human),207,['6T80'],6T80,E/F/G/H=197-207
...,...,...,...,...,...,...,...,...,...,...
7476,P12956,reviewed,XRCC6_HUMAN,X-ray repair cross-complementing protein 6 (EC...,XRCC6 G22P1,Homo sapiens (Human),609,"['1JEQ', '7SGL', '8AG4', '8AG5', '7AXZ', '6ERG...",1JEQ,A=1-609
7501,P63104,reviewed,1433Z_HUMAN,14-3-3 protein zeta/delta (Protein kinase C in...,YWHAZ,Homo sapiens (Human),245,"['4ZDR', '8AH2', '6ZFD', '6ZFG', '7Q16', '3RDH...",4ZDR,A/B=1-230
7524,Q6NZY4,reviewed,ZCHC8_HUMAN,Zinc finger CCHC domain-containing protein 8 (...,ZCCHC8,Homo sapiens (Human),707,"['7S7C', '7S7B', '7Z4Y', '7Z4Z', '6C90', '5LXR...",7S7C,B/F=1-707
7603,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT...",8DV1,D=218-449


In [77]:
comb_df.loc[comb_df['Entry'] == "P06213"]

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain
3249,P06213,reviewed,INSR_HUMAN,Insulin receptor (IR) (EC 2.7.10.1) (CD antige...,INSR,Homo sapiens (Human),1382,"['6PXV', '7YQ3', '4ZXB', '7QID', '6SOF', '7MQR...",6PXV,A/C=28-1382


In [78]:
comb_df.to_excel("Uniport_human_AF2_Ranked_Chains_Filtered.xlsx", index=False)

In [83]:
pd.read_excel("Uniport_human_AF2_Ranked_Chains_Filtered.xlsx")

,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,PDB_Files_Ordered,BestRanked,Chain
0,Q9NQ94,reviewed,A1CF_HUMAN,APOBEC1 complementation factor (APOBEC1-stimul...,A1CF ACF ASP,Homo sapiens (Human),594,['2CPD'],2CPD,A=223-308
1,Q8N5Z0,reviewed,AADAT_HUMAN,Kynurenine/alpha-aminoadipate aminotransferase...,AADAT KAT2 KYAT2,Homo sapiens (Human),425,"['6D0A', '4GEB', '4GE9', '4GE7', '6T8Q', '5EFS...",6D0A,A=1-425
2,Q2M2I8,reviewed,AAK1_HUMAN,AP2-associated protein kinase 1 (EC 2.7.11.1) ...,AAK1 KIAA1048,Homo sapiens (Human),961,"['5TE0', '4WSQ', '5L4Q']",5TE0,A=27-365
3,Q9H7C9,reviewed,AAMDC_HUMAN,Mth938 domain-containing protein (Adipogenesis...,AAMDC C11orf67 PTD015,Homo sapiens (Human),122,"['2Q4Q', '2AB1']",2Q4Q,A/B=2-122
4,Q16613,reviewed,SNAT_HUMAN,Serotonin N-acetyltransferase (Serotonin acety...,AANAT SNAT,Homo sapiens (Human),207,['6T80'],6T80,E/F/G/H=197-207
...,...,...,...,...,...,...,...,...,...,...
7601,P12956,reviewed,XRCC6_HUMAN,X-ray repair cross-complementing protein 6 (EC...,XRCC6 G22P1,Homo sapiens (Human),609,"['1JEQ', '7SGL', '8AG4', '8AG5', '7AXZ', '6ERG...",1JEQ,A=1-609
7602,P63104,reviewed,1433Z_HUMAN,14-3-3 protein zeta/delta (Protein kinase C in...,YWHAZ,Homo sapiens (Human),245,"['4ZDR', '8AH2', '6ZFD', '6ZFG', '7Q16', '3RDH...",4ZDR,A/B=1-230
7603,Q6NZY4,reviewed,ZCHC8_HUMAN,Zinc finger CCHC domain-containing protein 8 (...,ZCCHC8,Homo sapiens (Human),707,"['7S7C', '7S7B', '7Z4Y', '7Z4Z', '6C90', '5LXR...",7S7C,B/F=1-707
7604,P0DOX5,reviewed,IGG1_HUMAN,Immunoglobulin gamma-1 heavy chain (Immunoglob...,NaN,Homo sapiens (Human),449,"['8DV1', '8DV2', '7CZV', '7CZU', '5O4E', '6BFT...",8DV1,D=218-449
